In [20]:
import cnvpytor
import subprocess

print(cnvpytor.__version__)

1.3.1


In [21]:
filename = "test_with_del__5.fa"
command = f"cat {filename} | wc -l"
result = subprocess.check_output(command, shell=True)
line_count = int(result.strip())
line_count

0

In [22]:
read_count = line_count * 50 * 30 / 202
read_count = round(read_count)
read_count

0

In [23]:
command = f"wgsim -d 300 -N {read_count} -1 101 -2 101 {filename} frag1Copy_test.fastq frag2Copy_test.fastq"
result = subprocess.check_output(command, shell=True)
print(result)

b''


[wgsim] seed = 1706735683
[wgsim_core] calculating the total length of the reference sequence...
[wgsim_core] 0 sequences, total length: 0


In [24]:
# bowtie related stuff
!bowtie2-build reference.fa bias
!bowtie2 -p 4 -k 15 --no-mixed -x bias -1 frag1Copy_test.fastq -2 frag2Copy_test.fastq -S biasOut.sam

Settings:
  Output files: "bias.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  reference.fa
Building a SMALL index
Reading reference sizes
  Time reading reference sizes: 00:00:00
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:01
bmax according to bmaxDivN setting: 12826141
Using parameters --bmax 9619606 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 9619606 --dcv 1024
Constructing suffix-array element

In [25]:
# sam to bam file conversion
!samtools sort biasOut.sam -o biasOut_sorted.bam
!samtools index biasOut_sorted.bam

In [26]:
app = cnvpytor.Root("test.pytor", create=True, max_cores=8)

In [27]:
app.rd(["biasOut_sorted.bam"])

In [28]:
app.calculate_histograms([100, 1000, 10000, 100000, 1000000, 10000000])

In [29]:
app.partition([100, 1000, 10000, 100000, 1000000, 10000000])

In [30]:
calls = app.call([100, 1000, 10000, 100000, 1000000, 10000000], print_calls=True)

duplication	chr22:643201-644800	1600	1.4461	1.677376e-05	3.298560e-61	1.000000e+00	1.000000e+00	0.0000	1.0000	0
duplication	chr22:1306101-1325000	18900	1.3608	0.000000e+00	4.216876e-245	0.000000e+00	3.059357e-218	0.0000	1.0000	0
duplication	chr22:2926201-2963000	36800	3.0492	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:4546401-4713300	166900	1.6215	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:6655001-6661800	6800	2.3641	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:6848301-6854600	6300	1.6271	0.000000e+00	3.251713e-255	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:8039401-8048000	8600	1.8828	0.000000e+00	9.181974e-67	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:9424001-9461600	37600	1.4468	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:13230601-13395900	165300	1.3699	0.000000e+00	0.000000e+00	0.0000

In [31]:
for bin_size in calls:
    print("Bin size: ", bin_size)

Bin size:  100
Bin size:  1000
Bin size:  10000
Bin size:  100000
Bin size:  1000000
Bin size:  10000000


In [32]:
print(f"{filename=}, Histogram Size: 100")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[100]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__5.fa', Histogram Size: 100
type,chromosome_name,start,end,size,cnv_level,q0
duplication,chr22,643201,644800,1600,1.4460920920181561,0.0
duplication,chr22,1306101,1325000,18900,1.3608103353224403,0.0
duplication,chr22,2926201,2963000,36800,3.049165741050609,0.0
duplication,chr22,4546401,4713300,166900,1.621531950273017,0.0
duplication,chr22,6655001,6661800,6800,2.364065615091651,0.0
duplication,chr22,6848301,6854600,6300,1.6270802151764985,0.0
duplication,chr22,8039401,8048000,8600,1.8828497553058248,0.0
duplication,chr22,9424001,9461600,37600,1.4468310095558117,0.0
duplication,chr22,13230601,13395900,165300,1.369891445031509,0.0
duplication,chr22,13449001,13453700,4700,1.2887331273238247,0.0
duplication,chr22,13480901,13512800,31900,4.1247824554979955,0.0
duplication,chr22,15242601,15246400,3800,2.937446992769944,0.0
duplication,chr22,18663101,18664600,1500,1.5356847013267585,0.0
duplication,chr22,18664801,18665900,1100,1.5185226138059271,0.0
duplication,chr22,

In [33]:
print(f"{filename=}, Histogram Size: 1000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[1000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__5.fa', Histogram Size: 1000
type,chromosome_name,start,end,size,cnv_level,q0
duplication,chr22,1306001,1325000,19000,1.3596321382307355,0.0
duplication,chr22,2926001,2963000,37000,3.0391809086707338,0.0
duplication,chr22,4546001,4713000,167000,1.6209300862249836,0.0
duplication,chr22,6655001,6662000,7000,2.329798581236296,0.0
duplication,chr22,8040001,8048000,8000,1.8962044537247174,0.0
duplication,chr22,9424001,9462000,38000,1.442908306331749,0.0
duplication,chr22,13231001,13396000,165000,1.3698390660490365,0.0
duplication,chr22,13481001,13513000,32000,4.113375420631604,0.0
duplication,chr22,18705001,18881000,176000,2.7938462552290826,0.0001409582497203033
duplication,chr22,19259001,19268000,9000,3.2843019031245286,0.0
duplication,chr22,20634001,20655000,21000,1.3815972863310273,0.00028363430084144843
duplication,chr22,21248001,21350000,102000,1.5260487740242128,0.00017301636734835116
duplication,chr22,21351001,21382000,31000,1.4827390256197444,5.7608996220849

In [34]:
print(f"{filename=}, Histogram Size: 10,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[10000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__5.fa', Histogram Size: 10,000
type,chromosome_name,start,end,size,cnv_level,q0
duplication,chr22,4550001,4710000,160000,1.6070758588819976,0.0
duplication,chr22,13230001,13400000,170000,1.348776844715605,0.0
duplication,chr22,18710001,18880000,170000,2.7661266903213226,0.0001454790384179212
duplication,chr22,21250001,21380000,130000,1.5010125271637706,0.00014378730330982096
duplication,chr22,21530001,21670000,140000,1.350629022775494,0.0004606205013086456
duplication,chr22,25150001,25310000,160000,1.2948686709226875,0.00013440807596796651
duplication,chr22,28470001,28600000,130000,3.0706460963042446,4.2746651627199554e-05
duplication,chr22,30800001,30980000,180000,2.9497407508493128,2.598460412205768e-05
duplication,chr22,31150001,31360000,210000,1.4426486596781993,5.143403407770796e-05
duplication,chr22,31880001,32010000,130000,1.3009806523639595,0.00013177728995182093
duplication,chr22,32020001,32250000,230000,1.2970187165499476,8.574303178238783e-05
duplicat

In [35]:
print(f"{filename=}, Histogram Size: 100,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[100000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__5.fa', Histogram Size: 100,000
type,chromosome_name,start,end,size,cnv_level,q0
duplication,chr22,45700001,51300000,5600000,1.9571206751451697,7.261860778928292e-05


In [36]:
print(f"{filename=}, Histogram Size: 1,000,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[1000000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__5.fa', Histogram Size: 1,000,000
type,chromosome_name,start,end,size,cnv_level,q0


In [37]:
print(f"{filename=}, Histogram Size: 10,000,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[10000000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__5.fa', Histogram Size: 10,000,000
type,chromosome_name,start,end,size,cnv_level,q0


In [38]:
del app